**Author :  Ong Cheng Kei TP055620** <br>
**Description :**
<br>This file contains code to preprocess the USDA-FNDDS Nutrients Database.<br>The input file used is 2017-2018 FNDDS At A Glance - FNDDS Nutrient Values.xlsx and the output contains two files which are cleaned_fndds_nutrient_values.csv (tab separated) and cleaned_food_category.txt

In [1]:
from pprint import pprint

import pandas as pd

In [2]:
FNDDS_NUTRIENT_VALUES_PATH = (
    "./2017-2018 FNDDS At A Glance - FNDDS Nutrient Values.xlsx"
)

In [3]:
df_fndds_nutrient_values = pd.read_excel(FNDDS_NUTRIENT_VALUES_PATH, 0, skiprows=1)

In [4]:
df_fndds_nutrient_values.head()

,Food code,Main food description,WWEIA Category number,WWEIA Category description,Energy (kcal),Protein (g),Carbohydrate (g),"Sugars, total\n(g)","Fiber, total dietary (g)",Total Fat (g),...,20:1\n(g),22:1\n(g),18:2\n(g),18:3\n(g),18:4\n(g),20:4\n(g),20:5 n-3\n(g),22:5 n-3\n(g),22:6 n-3\n(g),Water\n(g)
0,11000000,"Milk, human",9602,Human milk,70,1.03,6.89,6.89,0.0,4.38,...,0.040,0.0,0.374,0.052,0.0,0.026,0.000,0.000,0.0,87.50
1,11100000,"Milk, NFS",1004,"Milk, reduced fat",51,3.34,4.87,4.89,0.0,1.99,...,0.002,0.0,0.069,0.007,0.0,0.003,0.000,0.001,0.0,89.04
2,11111000,"Milk, whole",1002,"Milk, whole",60,3.28,4.67,4.81,0.0,3.20,...,0.004,0.0,0.115,0.012,0.0,0.004,0.001,0.002,0.0,88.10
3,11111100,"Milk, low sodium, whole",1002,"Milk, whole",61,3.10,4.46,4.46,0.0,3.46,...,0.000,0.0,0.078,0.050,0.0,0.000,0.000,0.000,0.0,88.20
4,11111150,"Milk, calcium fortified, whole",1002,"Milk, whole",60,3.28,4.67,4.81,0.0,3.20,...,0.004,0.0,0.115,0.012,0.0,0.004,0.001,0.002,0.0,88.10


In [5]:
pprint(set(df_fndds_nutrient_values["WWEIA Category description"].tolist()))

{'Apple juice',
 'Apples',
 'Baby food: cereals',
 'Baby food: fruit',
 'Baby food: meat and dinners',
 'Baby food: snacks and sweets',
 'Baby food: vegetable',
 'Baby food: yogurt',
 'Baby juice',
 'Baby water',
 'Bacon',
 'Bagels and English muffins',
 'Bananas',
 'Bean, pea, legume dishes',
 'Beans, peas, legumes',
 'Beef, excludes ground',
 'Beer',
 'Biscuits, muffins, quick breads',
 'Blueberries and other berries',
 'Bottled water',
 'Broccoli',
 'Burgers (single code)',
 'Burritos and tacos',
 'Butter and animal fats',
 'Cabbage',
 'Cakes and pies',
 'Candy containing chocolate',
 'Candy not containing chocolate',
 'Carrots',
 'Cereal bars',
 'Cheese',
 'Cheese sandwiches (single code)',
 'Chicken patties, nuggets and tenders',
 'Chicken, whole pieces',
 'Chicken/turkey sandwiches (single code)',
 'Citrus fruits',
 'Citrus juice',
 'Coffee',
 'Cold cuts and cured meats',
 'Coleslaw, non-lettuce salads',
 'Cookies and brownies',
 'Corn',
 'Cottage/ricotta cheese',
 'Crackers, exc

In [6]:
def preprocess_wweia_category(value):
    if value == "Chicken, whole pieces":
        value = "Chicken in whole pieces"
    if value == "Chicken/turkey sandwiches (single code)":
        value = "Chicken sandwiches or turkey sandwiches"
    if value == "Coleslaw, non-lettuce salads":
        value = "Coleslaw or non-lettuce salads"
    if value == "Cottage/ricotta cheese":
        value = "Cottage cheese or ricotta cheese"
    if value == "Egg/breakfast sandwiches (single code)":
        value = "Egg sandwiches or breakfast sandwiches"
    if (
        value.__contains__("Flavored milk")
        or value.__contains__("Milk")
        or value.__contains__("Pasta sauces")
        or value.__contains__("White potatoes")
        or value.__contains__("Yogurt")
    ) and value.__contains__(","):
        tokens = value.split(",")
        value = tokens[-1] + " " + tokens[0]
    if value == "Fried rice and lo/chow mein":
        value = "Fried rice and lo mein or chow mein"

    value = value.replace(", excludes", " but excludes")
    value = value.replace(" (single code)", "")
    value = value.replace(",", " and")
    value = value.replace("/", " or ")
    value = value.lower()
    value = value.strip()

    return value


def preprocess_food_description(value):
    if (
        value
        == "Gyro sandwich (pita bread, beef, lamb, onion, condiments), with tomato and spread"
    ):
        value = (
            "Gyro sandwich with pita bread, beef, lamb, onion, tomato and condiments"
        )
    if value == "Dosa (Indian), plain":
        value = "Plain Indian Dosa"
    if value == "Dosa (Indian), with filling":
        value = "Indian Dosa with filling"
    if value == "Lefse (Norwegian)":
        value = "Norwegian Lefse"
    if value.__contains__("("):
        tokens = value.split("(")
        if tokens[-1].__contains__("%") or tokens[-1].__contains__("skim"):
            tokens.pop(-1)
        else:
            tokens[-1] = tokens[-1].replace(")", "")
            tokens[-1] = ", " + tokens[-1]
        value = "".join(tokens)
    value = value.replace("NFS", "Not Further Specified")
    value = value.replace("NS", "Not Specified")
    value = value.replace("/", " or ")
    value = value.lower()
    return value


def preprocess_nutrient_values(value):
    return value / 100


def clean_fndds_nutrient_values(df):
    maintain_columns = [
        "Main food description",
        "WWEIA Category description",
        "Energy (kcal)",
        "Protein (g)",
        "Carbohydrate (g)",
        "Total Fat (g)",
    ]
    new_df = df[maintain_columns]
    remove_categories = [
        "Not included in a food category",
        "Human milk",
        "Tap water",
        "Enhanced or fortified water",
        "Diet soft drinks",
        "Diet sport and energy drinks",
        "Flavored or carbonated water",
        "Smoothies and grain drinks",
        "Sport and energy drinks",
        "Protein and nutritional powders",
        "Nutritional beverages",
        "Nutrition bars",
        "Cereal bars",
        "Bottled water",
        "Milk shakes and other dairy drinks",
        "Fruit drinks",
        "Other fruit juice",
        "Other diet drinks",
        "Ready-to-eat cereal, higher sugar (>21.2g/100g)",
        "Ready-to-eat cereal, lower sugar (=<21.2g/100g)",
        "Coffee",
        "Citrus juice",
    ]
    new_df = new_df[
        ~new_df["WWEIA Category description"].isin(remove_categories)
        & ~new_df["WWEIA Category description"].str.contains("Baby")
        & ~new_df["WWEIA Category description"].str.contains("Formula")
    ]

    # Preprocessing remove noise from food category
    new_df["WWEIA Category description"] = new_df["WWEIA Category description"].apply(
        lambda x: preprocess_wweia_category(x)
    )

    # Preprocessing for main food description
    new_df["Main food description"] = new_df["Main food description"].apply(
        lambda x: preprocess_food_description(x)
    )

    new_df["Energy (kcal)"] = new_df["Energy (kcal)"].apply(
        lambda x: preprocess_nutrient_values(x)
    )
    new_df["Carbohydrate (g)"] = new_df["Carbohydrate (g)"].apply(
        lambda x: preprocess_nutrient_values(x)
    )
    new_df["Protein (g)"] = new_df["Protein (g)"].apply(
        lambda x: preprocess_nutrient_values(x)
    )
    new_df["Total Fat (g)"] = new_df["Total Fat (g)"].apply(
        lambda x: preprocess_nutrient_values(x)
    )

    return new_df

In [7]:
df_fndds_nutrient_values = clean_fndds_nutrient_values(df_fndds_nutrient_values)

In [8]:
cleaned_categories = set(
    df_fndds_nutrient_values["WWEIA Category description"].tolist()
)
cleaned_categories

{'apple juice',
 'apples',
 'bacon',
 'bagels and english muffins',
 'baked or boiled white potatoes',
 'bananas',
 'bean and pea and legume dishes',
 'beans and peas and legumes',
 'beef but excludes ground',
 'beer',
 'biscuits and muffins and quick breads',
 'blueberries and other berries',
 'broccoli',
 'burgers',
 'burritos and tacos',
 'butter and animal fats',
 'cabbage',
 'cakes and pies',
 'candy containing chocolate',
 'candy not containing chocolate',
 'carrots',
 'cheese',
 'cheese sandwiches',
 'chicken in whole pieces',
 'chicken patties and nuggets and tenders',
 'chicken sandwiches or turkey sandwiches',
 'citrus fruits',
 'cold cuts and cured meats',
 'coleslaw or non-lettuce salads',
 'cookies and brownies',
 'corn',
 'cottage cheese or ricotta cheese',
 'crackers but excludes saltines',
 'cream and cream substitutes',
 'cream cheese and sour cream and whipped cream',
 'dips and gravies and other sauces',
 'doughnuts and sweet rolls and pastries',
 'dried fruits',
 'e

In [11]:
with open("./cleaned_food_category.txt", "w") as file:
    file.writelines(sent + "\n" for sent in cleaned_categories)

In [12]:
df_fndds_nutrient_values.to_csv(
    "./cleaned_fndds_nutrient_values.csv", sep="\t", index=False, float_format="%.4f"
)